In [ ]:
# Step 1: Import Dependencies (already in your code)
import os
import pinecone
import openai
#import logging
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import Pinecone as PineconeVectorStore
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from pinecone import Pinecone, ServerlessSpec
from langchain.vectorstores import Pinecone as PineconeVectorStore
from langchain.chat_models import AzureChatOpenAI

In [ ]:
# Step 2: Read Pinecone and Azure OpenAI Environment Variables
os.environ["AZURE_OPENAI_API_KEY"] = "BjSM1Dwo5UZVvPUizHw8w0n8i7TM3fHIK3GjbeIYX5Z1nqffyiCBJQQJ99BBACYeBjFXJ3w3AAABACOGRhVh"
os.environ["AZURE_OPENAI_API_BASE"] = "https://khipus-aoai.openai.azure.com"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "text-embedding-ada-002"
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-05-15"

openai.api_key = os.environ["AZURE_OPENAI_API_KEY"]
openai.api_base = os.environ["AZURE_OPENAI_API_BASE"]
openai.api_type = "azure"
openai.api_version = os.environ["AZURE_OPENAI_API_VERSION"]

#os.environ["PINECONE_API_KEY"] = "pcsk_56sL2K_6uS61piqrBjZkoCem53GXgvmUDvZTajV59k1gUKxJitiHqTUPi5ho5m1WThbh6x"
#os.environ["PINECONE_ENV"] = "us-east-1-aws"  # Change as needed

In [5]:
# Step 3: Load your PDF and split into chunks
pdf_path = "./docs/corollacross_brochure.pdf"  # Adjust the file path if needed
loader = PyPDFLoader(pdf_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
print(f"Loaded {len(documents)} document(s) and split into {len(docs)} chunks.")

Loaded 23 document(s) and split into 78 chunks.


In [6]:
# Step 4: Initialize the Azure OpenAI embeddings object using LangChain.
embeddings = AzureOpenAIEmbeddings(
    openai_api_key=openai.api_key,
    azure_endpoint=openai.api_base,  # Use azure_endpoint instead of openai_api_base
    openai_api_version=openai.api_version,
    deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"]
)

In [7]:


# Step 5: Generate embeddings for each chunk
pinecone.Index = pinecone.data.index.Index

# Replace these values as needed
api_key = "pcsk_56sL2K_6uS61piqrBjZkoCem53GXgvmUDvZTajV59k1gUKxJitiHqTUPi5ho5m1WThbh6x" # Pinecone API key
index_name = "langchain-demo"

# Create an instance of the Pinecone class using the new API
pc = Pinecone(api_key=api_key)

# List indexes to check connectivity
print("Available indexes:", pc.list_indexes().names())

# Create (or connect to) your Pinecone index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # Must match the embedding dimension
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"  # Ensure this matches your Pinecone dashboard
        )
    )
    print(f"Created index: {index_name}")
else:
    print(f"Index '{index_name}' already exists.")




Available indexes: ['example-index', 'langchain-demo']
Index 'langchain-demo' already exists.


In [8]:
# Step 6 Create and store embeddings using the PineconeVectorStore
# Retrieve the index client
index = pc.Index(index_name)

# Create and store embeddings using the PineconeVectorStore
vector_store = PineconeVectorStore(
    index,         # The instance of pinecone.Index
    embeddings,    # Your initialized embeddings object (Azure OpenAI embeddings)
    text_key="text",  # Adjust if your documents use a different key
    namespace="default"
)

# Assuming 'docs' contains your document chunks
vector_store.add_documents(docs)

print("Embeddings have been successfully stored in Pinecone!")

C:\Users\sanikj\AppData\Local\Temp\ipykernel_28016\3052846429.py:6: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  vector_store = PineconeVectorStore(


Embeddings have been successfully stored in Pinecone!


In [ ]:
#import os
#from langchain.chat_models import AzureChatOpenAI

# Initialize the language model using Azure Chat OpenAI
llm = AzureChatOpenAI(
    temperature=0,
    openai_api_base=os.environ["AZURE_OPENAI_API_BASE"],
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    openai_api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-10-21"),
    deployment_name=os.environ.get("AZURE_OPENAI_GPT4_MODEL_NAME", "gpt-4o")
)

# Load the QA chain
chain = load_qa_chain(llm, chain_type="stuff")

# Define your query
query = "What is the engine size of the Toyota Corolla Cross?"
#What is the estimated fuel efficiency of the Corolla Cross Hybrid?

# Retrieve similar documents from the vector store (removed include_metadata)
docs = vector_store.similarity_search(query)

# Optionally, access metadata from the documents if needed
for doc in docs:
    print("Metadata:", doc.metadata)

# Get the answer from the chain
result = chain.run(input_documents=docs, question=query)

print(f"Answer: \n\n{result}")



c:\temp\building-qa-app-with-openai-pinecone-and-streamlit\venvrag\Lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://khipus-aoai.openai.azure.com to https://khipus-aoai.openai.azure.com/openai.
  warnings.warn(
c:\temp\building-qa-app-with-openai-pinecone-and-streamlit\venvrag\Lib\site-packages\langchain_community\chat_models\azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
c:\temp\building-qa-app-with-openai-pinecone-and-streamlit\venvrag\Lib\site-packages\langchain_community\chat_models\azure_openai.py:189: UserWarning: As of openai>=1.0.0, if `openai_api_

Metadata: {'creationdate': '2024-01-17T14:56:22-05:00', 'creator': 'Adobe InDesign 18.5 (Macintosh)', 'moddate': '2024-01-17T14:57:41-05:00', 'page': 2.0, 'page_label': '3', 'producer': 'Adobe PDF Library 17.0', 'source': './docs/corollacross_brochure.pdf', 'total_pages': 23.0, 'trapped': '/False'}
Metadata: {'creationdate': '2024-01-17T14:56:22-05:00', 'creator': 'Adobe InDesign 18.5 (Macintosh)', 'moddate': '2024-01-17T14:57:41-05:00', 'page': 2.0, 'page_label': '3', 'producer': 'Adobe PDF Library 17.0', 'source': './docs/corollacross_brochure.pdf', 'total_pages': 23.0, 'trapped': '/False'}
Metadata: {'creationdate': '2024-01-17T14:56:22-05:00', 'creator': 'Adobe InDesign 18.5 (Macintosh)', 'moddate': '2024-01-17T14:57:41-05:00', 'page': 2.0, 'page_label': '3', 'producer': 'Adobe PDF Library 17.0', 'source': './docs/corollacross_brochure.pdf', 'total_pages': 23.0, 'trapped': '/False'}
Metadata: {'creationdate': '2024-01-17T14:56:22-05:00', 'creator': 'Adobe InDesign 18.5 (Macintosh)'